In [ ]:

import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import random



# # Configuration constants
# URL = "https://www.randhawa.us/games/retailer/nyu.html"
# TARGET_TOTAL = 2000  # Total inventory
# WEEKS = 15           # Game duration in weeks
# SIMULATIONS = 2   # Number of simulation rounds to run
# WAIT_TIMEOUT = 2    # Maximum seconds to wait for elements

# # Button IDs/text for locating elements (adjust as needed based on actual page)
# BUTTONS = {
#     'restart': "practiceButton",       # Restart Game
#     'maintain': "maintainButton",      # Maintain Price
#     'md10': "tenButton",               # 10% Markdown
#     'md20': "twentyButton",            # 20% Markdown
#     'md40': "fortyButton"              # 40% Markdown
# }


# Implementation

# tRY 2

In [ ]:
# #!/usr/bin/env python3
# """
# Retailer Game Automation Script
# ----------------------------
# This script automates playing the Retailer Game 1,000 times using Selenium.
# It tracks detailed weekly results and final outcomes in separate CSV files.
# """

# import csv
# import os
# import time
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import TimeoutException, NoSuchElementException

# # Game URL
# GAME_URL = "https://www.randhawa.us/games/retailer/nyu.html"

# # Button IDs for various actions
# BUTTONS = {
#     'restart': "practiceButton",  # Restart Game
#     'maintain': "maintainButton", # Maintain Price
#     'md10': "tenButton",          # 10% Markdown
#     'md20': "twentyButton",       # 20% Markdown
#     'md40': "fortyButton"         # 40% Markdown
# }

# def creeping_strategy():
#     """
#     Generator function that implements a price markdown strategy.
    
#     This strategy uses an adaptive approach based on the sales pace
#     and remaining weeks to try to clear inventory by the end of the game.
    
#     Yields:
#         str: A key from BUTTONS dictionary indicating the next action
#     """
#     week = 1
#     last_action = None
    
#     # First action (needs to be primed)
#     action = yield None
    
#     while week <= 14:
#         # Receive cumulative sales from the main loop
#         cum_sales = yield action
        
#         # Calculate the remaining inventory
#         remaining_inventory = 2000 - cum_sales
        
#         # Calculate weeks left
#         weeks_left = 15 - week
        
#         if weeks_left == 0:
#             # No more decisions to make
#             action = 'maintain'
#             continue
            
#         # Calculate the sales rate needed to clear inventory
#         needed_weekly_sales_rate = remaining_inventory / weeks_left if weeks_left > 0 else float('inf')
        
#         # Calculate the current weekly sales rate (average of past weeks)
#         current_weekly_sales_rate = cum_sales / week if week > 0 else 0
        
#         # Calculate the ratio of current sales rate to needed rate
#         sales_rate_ratio = current_weekly_sales_rate / needed_weekly_sales_rate if needed_weekly_sales_rate > 0 else float('inf')
        
#         # Logic for making decisions
#         if sales_rate_ratio >= 1.2:
#             # Sales are well ahead of pace - maintain price
#             action = 'maintain'
#         elif sales_rate_ratio >= 1.0:
#             # Sales are on pace - maintain or small markdown
#             action = 'maintain' if week < 8 else 'md10'
#         elif sales_rate_ratio >= 0.8:
#             # Slightly behind pace - 10% markdown
#             action = 'md10'
#         elif sales_rate_ratio >= 0.6:
#             # Moderately behind - 20% markdown
#             action = 'md20'
#         else:
#             # Significantly behind - 40% markdown
#             action = 'md40'
        
#         # Apply price adjustment restrictions (once price is reduced, cannot go back up)
#         if last_action == 'md10' and action == 'maintain':
#             action = 'md10'
#         elif last_action in ['md20', 'md40'] and action in ['maintain', 'md10']:
#             action = last_action
#         elif last_action == 'md40':
#             action = 'md40'  # Once at 40%, stay there
        
#         last_action = action
#         week += 1


# def play_simulation(driver, sim_num, week_writer, outcome_writer, strategy_generator):
#     """
#     Play one complete simulation of the Retailer Game
    
#     Args:
#         driver: Selenium WebDriver instance
#         sim_num: Simulation number (for tracking)
#         week_writer: CSV writer for weekly detailed data
#         outcome_writer: CSV writer for final outcomes
#         strategy_generator: Generator function that yields next action
#     """
#     try:
#         # Navigate to the game URL if not already there
#         if driver.current_url != GAME_URL:
#             driver.get(GAME_URL)
#             time.sleep(1)  # Brief pause for page load
        
#         # Click the Restart Game button
#         restart_button = WebDriverWait(driver, 10).until(
#             EC.element_to_be_clickable((By.ID, BUTTONS['restart']))
#         )
#         restart_button.click()
        
#         # Initialize tracking variables
#         week = 1
#         inventory = 2000
#         disallowed_actions = set()
#         total_revenue = 0.0
        
#         # Prime the strategy generator
#         strategy = strategy_generator()
#         next(strategy)  # Prime the generator
#         action_key = strategy.send(None)  # Get first action
        
#         # Main game loop
#         while week <= 15 and inventory > 0:
#             # Wait for the table to load/update
#             table = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.ID, "result-table"))
#             )
            
#             # Extract data from the table - all rows are in the thead section
#             rows = table.find_elements(By.XPATH, ".//thead/tr")
            
#             # Skip the header row (first row)
#             if len(rows) < 2:  # No data rows yet
#                 current_week = 0
#                 current_price = 60.0  # Initial price (based on provided example)
#                 current_sales = 0
#                 current_inventory = 2000
#             else:
#                 last_row = rows[-1]  # Last row has the current week's data
#                 cells = last_row.find_elements(By.TAG_NAME, "td")
                
#                 # Parse the data from the cells
#                 current_week = int(cells[0].text) if cells[0].text else 0
#                 current_price = float(cells[1].text) if cells[1].text else 0.0
#                 current_sales = int(cells[2].text) if cells[2].text else 0
#                 current_inventory = int(cells[3].text) if cells[3].text else 0
            
#             # Update tracking variables
#             inventory = current_inventory
#             total_revenue += current_price * current_sales
            
#             # Write week details to CSV
#             week_writer.writerow([
#                 sim_num, 
#                 current_week, 
#                 current_price, 
#                 current_sales, 
#                 current_inventory
#             ])
            
#             # Check if the game is over
#             if week == 15 or inventory == 0:
#                 break
            
#             # Get next action from strategy generator
#             cumulative_sales = 2000 - inventory
#             action_key = strategy.send(cumulative_sales)
            
#             # Apply the action restrictions
#             if 'maintain' in disallowed_actions and action_key == 'maintain':
#                 action_key = 'md10'  # Default to 10% if maintain is disallowed
#             if 'md10' in disallowed_actions and action_key == 'md10':
#                 action_key = 'md20'  # Default to 20% if 10% is disallowed
            
#             # Click the button for the chosen action
#             if action_key in BUTTONS and action_key not in disallowed_actions:
#                 try:
#                     button = WebDriverWait(driver, 5).until(
#                         EC.element_to_be_clickable((By.ID, BUTTONS[action_key]))
#                     )
#                     button.click()
                    
#                     # Update disallowed actions based on current action
#                     if action_key == 'md10':
#                         disallowed_actions.add('maintain')
#                     elif action_key == 'md20':
#                         disallowed_actions.add('maintain')
#                         disallowed_actions.add('md10')
#                     elif action_key == 'md40':
#                         # After 40%, no further actions allowed
#                         disallowed_actions.update(['maintain', 'md10', 'md20', 'md40'])
#                 except (TimeoutException, NoSuchElementException):
#                     print(f"Button {action_key} not clickable in simulation {sim_num}, week {week}")
            
#             # Increment week counter
#             week += 1
            
#             # Small wait to ensure game state updates
#             time.sleep(0.5)
        
#         # Extract final results
#         try:
#             user_revenue = WebDriverWait(driver, 5).until(
#                 EC.presence_of_element_located((By.ID, "rev"))
#             ).text
#             perfect_revenue = driver.find_element(By.ID, "perfect").text
#             percentage_diff = driver.find_element(By.ID, "percentage").text.replace(", Difference: ", "")
            
#             # Clean up the revenue values (remove $ and commas)
#             user_revenue = user_revenue.replace("$", "").replace(",", "")
#             perfect_revenue = perfect_revenue.replace("$", "").replace(",", "")
            
#             # Write outcome to CSV
#             outcome_writer.writerow([
#                 sim_num,
#                 user_revenue,
#                 perfect_revenue,
#                 percentage_diff
#             ])
            
#         except (TimeoutException, NoSuchElementException) as e:
#             print(f"Error extracting final results for simulation {sim_num}: {e}")
#             # Write whatever data we have
#             outcome_writer.writerow([sim_num, total_revenue, "N/A", "N/A"])
    
#     except Exception as e:
#         print(f"Error in simulation {sim_num}: {e}")


# def main():
#     """
#     Main function to run the automation
#     """
#     # Set up Chrome options for headless browser
#     chrome_options = Options()
#     #chrome_options.add_argument("--headless")
#     chrome_options.add_argument("--disable-gpu")
#     chrome_options.add_argument("--window-size=1920x1080")
#     chrome_options.add_argument("--no-sandbox")
    
#     # Initialize WebDriver
#     driver = webdriver.Chrome(options=chrome_options)
    
#     try:
#         # Create output directory if it doesn't exist
#         os.makedirs("results", exist_ok=True)
        
#         # Set up CSV files
#         with open("results/week_detail.csv", 'w', newline='') as week_file, \
#              open("results/outcome.csv", 'w', newline='') as outcome_file:
            
#             # Create CSV writers
#             week_writer = csv.writer(week_file)
#             outcome_writer = csv.writer(outcome_file)
            
#             # Write headers
#             week_writer.writerow([
#                 "Simulation Number", "Week", "Price", "Sales", "Remaining Inventory"
#             ])
#             outcome_writer.writerow([
#                 "Simulation Number", "Your revenue", "Perfect foresight strategy", "Difference (%)"
#             ])
            
#             # Run simulations
#             total_simulations = 2
#             for sim_num in range(1, total_simulations + 1):
#                 print(f"Running simulation {sim_num} of {total_simulations}")
#                 play_simulation(
#                     driver, 
#                     sim_num, 
#                     week_writer, 
#                     outcome_writer,
#                     creeping_strategy
#                 )
                
#                 # Optional: Add a small delay between simulations
#                 time.sleep(0.5)
                
#         print("All simulations complete!")
    
#     finally:
#         # Always close the driver
#         driver.quit()


# if __name__ == "__main__":
#     main()

Running simulation 1 of 2
Running simulation 2 of 2
Button md20 not clickable in simulation 2, week 2
All simulations complete!


# Atttempt 3

## Combo creation

In [2]:
from selenium import webdriver
import pandas as pd
from re import sub
import timeit
import itertools

# Create a list of all possible combos

x = [60, 54, 48, 36]
combos = []

for i in itertools.product(x,repeat=14):
    if i[0] >= i[1] >= i[2] >= i[3] >= i[4] >= i[5] >= i[6]>= i[7] >= i[8]>= i[9] >= i[10] >= i[11] >= i[12] >= i[13]:
        combos.append(i)

check = pd.DataFrame(combos)

combos_list = []

for i in combos:
    combos_list.append(list(i))

# First step in each combo is always 60

for i in combos_list:
    i.insert(0,60)

# Assign an ID to each combo

for i in range(len(combos_list)):
    combos_list[i].insert(0,i)

combos_df = pd.DataFrame(combos_list)

combos_df = combos_df.rename(columns={0:"combo_number"})

print(combos_df)

     combo_number   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
0               0  60  60  60  60  60  60  60  60  60  60  60  60  60  60  60
1               1  60  60  60  60  60  60  60  60  60  60  60  60  60  60  54
2               2  60  60  60  60  60  60  60  60  60  60  60  60  60  60  48
3               3  60  60  60  60  60  60  60  60  60  60  60  60  60  60  36
4               4  60  60  60  60  60  60  60  60  60  60  60  60  60  54  54
..            ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
675           675  60  48  48  48  48  36  36  36  36  36  36  36  36  36  36
676           676  60  48  48  48  36  36  36  36  36  36  36  36  36  36  36
677           677  60  48  48  36  36  36  36  36  36  36  36  36  36  36  36
678           678  60  48  36  36  36  36  36  36  36  36  36  36  36  36  36
679           679  60  36  36  36  36  36  36  36  36  36  36  36  36  36  36

[680 rows x 16 columns]


## Transform to action

In [5]:
import pandas as pd

def price_to_action(price_df):
    # Define the mapping from price change to action
    def get_action(prev_price, current_price):
        if current_price == prev_price:
            return 'A'  # Maintain price
        elif current_price == 54:
            return 'B'  # 10% markdown
        elif current_price == 48:
            return 'C'  # 20% markdown
        elif current_price == 36:
            return 'D'  # 40% markdown
        else:
            return '?'  # Unknown price

    # Copy the combo_number column
    action_df = pd.DataFrame()
    action_df['combo_number'] = price_df['combo_number']

    # Convert price values to actions
    for col_idx, col in enumerate(price_df.columns[1:], start=1):
        week = col  # Week number as column name
        if col_idx == 1:
            # First week: always maintain list price
            action_df[week] = 'A'
        else:
            prev_week = price_df.columns[col_idx - 1]
            action_df[week] = [
                get_action(prev, curr)
                for prev, curr in zip(price_df[prev_week], price_df[week])
            ]

    return action_df

# Assuming your original DataFrame is called `df`
action_df = price_to_action(combos_df)

# Print or save the transformed DataFrame
print(action_df.head())
action_df.to_csv("action_df.csv", index=False)


   combo_number  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
0             0  A  A  A  A  A  A  A  A  A  A  A  A  A  A  A
1             1  A  A  A  A  A  A  A  A  A  A  A  A  A  A  B
2             2  A  A  A  A  A  A  A  A  A  A  A  A  A  A  C
3             3  A  A  A  A  A  A  A  A  A  A  A  A  A  A  D
4             4  A  A  A  A  A  A  A  A  A  A  A  A  A  B  A


In [ ]:
display(action_df)

,combo_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A
1,1,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B
2,2,A,A,A,A,A,A,A,A,A,A,A,A,A,A,C
3,3,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D
4,4,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,675,A,C,A,A,A,D,A,A,A,A,A,A,A,A,A
676,676,A,C,A,A,D,A,A,A,A,A,A,A,A,A,A
677,677,A,C,A,D,A,A,A,A,A,A,A,A,A,A,A
678,678,A,C,D,A,A,A,A,A,A,A,A,A,A,A,A


In [2]:
import pandas as pd

action_df = pd.read_csv("action_df.csv")


print(action_df)

     combo_number  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
0               0  A  A  A  A  A  A  A  A  A  A  A  A  A  A  A
1               1  A  A  A  A  A  A  A  A  A  A  A  A  A  A  B
2               2  A  A  A  A  A  A  A  A  A  A  A  A  A  A  C
3               3  A  A  A  A  A  A  A  A  A  A  A  A  A  A  D
4               4  A  A  A  A  A  A  A  A  A  A  A  A  A  B  A
..            ... .. .. .. .. .. .. .. .. .. .. .. .. .. .. ..
675           675  A  C  A  A  A  D  A  A  A  A  A  A  A  A  A
676           676  A  C  A  A  D  A  A  A  A  A  A  A  A  A  A
677           677  A  C  A  D  A  A  A  A  A  A  A  A  A  A  A
678           678  A  C  D  A  A  A  A  A  A  A  A  A  A  A  A
679           679  A  D  A  A  A  A  A  A  A  A  A  A  A  A  A

[680 rows x 16 columns]


# Simulation Run

### V3

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import timeit
import re
import time

def run_simulation(browser, combo_row, simulation_number):
    """
    Run a single simulation with the given combo
    
    Parameters:
    - browser: WebDriver instance
    - combo_row: DataFrame row containing the combo actions
    - simulation_number: Unique identifier for this simulation
    
    Returns:
    - week_data: List of weekly results
    - outcome_data: Final outcome data
    """
    wait = WebDriverWait(browser, 10)
    
    # Map letters to button actions
    action_map = {
        'A': 'maintainButton',  # Maintain price (60)
        'B': 'tenButton',       # 10% discount (54)
        'C': 'twentyButton',    # 20% discount (48)
        'D': 'fortyButton'      # 40% discount (36)
    }
    
    # Click the button to start a new game
    try:
        new_game_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
        new_game_button.click()
    except:
        # If we can't find the button, reload the page
        browser.refresh()
        wait = WebDriverWait(browser, 10)
        new_game_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
        new_game_button.click()
    
    # Always use 100 as starting inventory (you can modify this if needed)
    inv_dropdown = wait.until(EC.element_to_be_clickable((By.ID, "startingInventory")))
    for option in browser.find_elements(By.TAG_NAME, "option"):
        if option.text == "100":
            option.click()
            break
    
    # Click "Start Game" button
    start_game_button = wait.until(EC.element_to_be_clickable((By.ID, "start-button")))
    start_game_button.click()
    
    combo_id = combo_row['combo_number']
    week_data = []
    
    # Play through each week according to the combo actions
    for week in range(1, 16):  # Weeks 1-15
        # Get the action letter for this week
        action_letter = combo_row[week]
        button_id = action_map[action_letter]
        
        # Click the corresponding button
        button = wait.until(EC.element_to_be_clickable((By.ID, button_id)))
        button.click()
        
        # Wait briefly for results to update
        time.sleep(0.3)
        
        # Extract data from the results table
        try:
            result_rows = browser.find_elements(By.XPATH, '//table[@id="result-table"]//tr')
            for row in result_rows[1:]:  # Skip header row
                tds = row.find_elements(By.TAG_NAME, 'td')
                if tds:
                    week_num = int(tds[0].text)
                    price = int(tds[1].text)
                    sales = int(tds[2].text)
                    remaining = int(tds[3].text)
                    
                    # Add to week detail data
                    week_data.append([
                        simulation_number,
                        combo_id,
                        week_num,
                        price,
                        sales,
                        remaining
                    ])
        except Exception as e:
            print(f"Error extracting data for week {week}: {e}")
    
    # After all weeks are done, capture final outcomes
    try:
        revenue = int(re.sub(r'[^\d.]', '', browser.find_element(By.ID, "rev").text))
        perfect = int(re.sub(r'[^\d.]', '', browser.find_element(By.ID, "perfect").text))
        difference_pct = ((perfect - revenue) / perfect) * 100 if perfect > 0 else 0
        
        outcome_data = [
            simulation_number,
            combo_id,
            revenue,
            perfect,
            difference_pct
        ]
    except Exception as e:
        print(f"Error capturing final outcome: {e}")
        outcome_data = [simulation_number, combo_id, 0, 0, 0]
    
    return week_data, outcome_data

def main():
    """Main function to run all simulations"""
    # Read the action DataFrame
    action_df = pd.read_csv('action_df.csv')
    
    # Setup Chrome options
    chrome_options = Options()
    # Set up Chrome options as requested
    # chrome_options.add_argument("--headless")  # Uncomment to run in headless mode
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--no-sandbox")
    
    # Initialize the browser using ChromeDriverManager (no explicit driver path needed)
    browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    browser.get("http://www.randhawa.us/games/retailer/nyu.html")
    
    # Wait for page to load
    wait = WebDriverWait(browser, 10)
    
    # Prepare data structures for results
    all_week_data = []
    all_outcome_data = []
    
    # Save frequency - after how many simulations to save interim results
    save_frequency = 10
    
    # Start timing
    start = timeit.default_timer()
    
    try:
        # Loop through each combo in the action_df
        for index, combo_row in action_df.iterrows():
            simulation_number = index + 1
            print(f"Running simulation {simulation_number}/{len(action_df)}: Combo {combo_row['combo_number']}")
            
            # Run the simulation and get results
            week_data, outcome_data = run_simulation(browser, combo_row, simulation_number)
            
            # Append results to our data collections
            all_week_data.extend(week_data)
            all_outcome_data.append(outcome_data)
            
            # Save interim results periodically
            if simulation_number % save_frequency == 0:
                print(f"Saving interim results after {simulation_number} simulations...")
                
                # Create and save week detail DataFrame
                week_df = pd.DataFrame(
                    all_week_data, 
                    columns=["Simulation Number", "comboID", "Week", "Price", "Sales", "Remaining Inventory"]
                )
                week_df.to_csv(f"week_detail_interim_{simulation_number}.csv", index=False)
                
                # Create and save outcome DataFrame
                outcome_df = pd.DataFrame(
                    all_outcome_data, 
                    columns=["Simulation Number", "comboID", "Your revenue", "Perfect foresight strategy", "Difference (%)"]
                )
                outcome_df.to_csv(f"outcome_interim_{simulation_number}.csv", index=False)
            
    except Exception as e:
        print(f"Simulation error: {e}")
    finally:
        # Close the browser
        browser.quit()
        
        # Calculate elapsed time
        stop = timeit.default_timer()
        print(f'Total simulation time: {stop - start:.2f} seconds')
        
        # Create final DataFrames and save to CSV
        week_df = pd.DataFrame(
            all_week_data, 
            columns=["Simulation Number", "comboID", "Week", "Price", "Sales", "Remaining Inventory"]
        )
        week_df.to_csv("week_detail.csv", index=False)
        
        outcome_df = pd.DataFrame(
            all_outcome_data, 
            columns=["Simulation Number", "comboID", "Your revenue", "Perfect foresight strategy", "Difference (%)"]
        )
        outcome_df.to_csv("outcome.csv", index=False)
        
        print("Simulation completed. Results saved to week_detail.csv and outcome.csv")

if __name__ == "__main__":
    main()

Running simulation 1/680: Combo 0
Simulation error: Message: 
Stacktrace:
	GetHandleVerifier [0x007CFC53+61635]
	GetHandleVerifier [0x007CFC94+61700]
	(No symbol) [0x005F05D3]
	(No symbol) [0x0063899E]
	(No symbol) [0x00638D3B]
	(No symbol) [0x00680E12]
	(No symbol) [0x0065D2E4]
	(No symbol) [0x0067E61B]
	(No symbol) [0x0065D096]
	(No symbol) [0x0062C840]
	(No symbol) [0x0062D6A4]
	GetHandleVerifier [0x00A54573+2701795]
	GetHandleVerifier [0x00A4FCF6+2683238]
	GetHandleVerifier [0x00A6AA3E+2793134]
	GetHandleVerifier [0x007E6915+155013]
	GetHandleVerifier [0x007ECFFD+181357]
	GetHandleVerifier [0x007D74A8+92440]
	GetHandleVerifier [0x007D7650+92864]
	GetHandleVerifier [0x007C2040+5296]
	BaseThreadInitThunk [0x76077BA9+25]
	RtlInitializeExceptionChain [0x7725C36B+107]
	RtlClearBits [0x7725C2EF+191]

Total simulation time: 12.89 seconds
Simulation completed. Results saved to week_detail.csv and outcome.csv


In [19]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))